###### Content under Creative Commons Attribution license CC-BY 4.0, code under MIT license (c) 2019 Daniel Koehn, based on (c)2018 L.A. Barba, G.F. Forsyth [CFD Python](https://github.com/barbagroup/CFDPython#cfd-python), (c)2014 L.A. Barba, I. Hawke, B. Knaepen [Practical Numerical Methods with Python](https://github.com/numerical-mooc/numerical-mooc#practical-numerical-methods-with-python), also under CC-BY.

In [1]:
from IPython.core.display import HTML
css_file = '../style/custom.css'
HTML(open(css_file, 'r').read())

# 1D Heat Conduction Example II: Cooling of a half-space

In this exercise we will focus on another problem in geophysics, which can be described by the 1D heat conduction equation: the cooling of a half-space. By solving this problem, we can understand the generation and cooling of the oceanic lithosphere on earth.

## Cooling of the Oceanic Lithosphere

The rigid outer shell of the earth, the [lithosphere](https://en.wikipedia.org/wiki/Lithosphere), is generated at the [**mid-ocean ridges (MOR)**](https://en.wikipedia.org/wiki/Mid-ocean_ridge) from molten mantle material. Due to [ridge push](https://en.wikipedia.org/wiki/Ridge_push) and [slab pull](https://en.wikipedia.org/wiki/Slab_pull) the new oceanic lithosphere is spreading away from the MORs. 

![MOR](./figures/MORB.jpg)
.
#### Figure 1. Generation and cooling of the oceanic lithosphere.

Neglecting heat transport by advection and restrict heat conduction to the vertical direction $z$, we can describe the cooling of the oceanic lithosphere by the 1D heat conduction equation:

$$
\begin{equation}
\frac{\partial T}{\partial t} = \alpha \frac{\partial^2 T}{\partial z^2} \notag
\end{equation}
$$

Here, $\alpha$ is the thermal diffusivity and temperature $T$. 

Let's take a deeper look into the problem.

##### Exercise 1

Lets assume that we have a partially molten half-space at the MOR with a constant temperature $T_{init} = 1300^oC$ down to a depth $z_1 = 250\;km$. The half-space is cooled by the lower surface temperature $T_{surf}=20^oC$ at $z_0 = 0\; km$.

Additionally, the [thermal diffusivity](http://en.wikipedia.org/wiki/Thermal_diffusivity) of the mantle rock is $\alpha=1\times10^{-6} {\rm m}^2/{\rm s}$. How will the temperature evolve in the lithosphere and upper mantle as a function of time? We can compare the numerical solution of the problem with an analytical solution:

\begin{equation}
T(z,t) = T_{init} + (T_{surf} - T_{init})\; \text{erfc}\biggl(\frac{z}{2\sqrt{\alpha t}}\biggr) \tag{1}
\end{equation}

where $\text{erfc}$ denotes the **Complementary Error Function**. The `erfc` can be computed using the `SciPy` package. For more details of the implementation, I refer to Prof. Ecosia.

In this exercise, we want to compute the temperature profiles $T(z,t)$ of the cooled half-space after $t=10\;\text{Ma}$, $t=60\;\text{Ma}$ and $t=100\;\text{Ma}$, respectively. 



As usual, start by importing some libraries and setting up the discretization.

In [ ]:
import numpy
from matplotlib import pyplot
from scipy import special
%matplotlib inline

In [ ]:
# Set the font family and size to use for Matplotlib figures.
pyplot.rcParams['font.family'] = 'serif'
pyplot.rcParams['font.size'] = 16

We'll begin by defining a few parameters ...

- depth of the half-space $L = 250\; \text{km}$
- a spatial grid with $nz = 100\; \text{points}$ in z-direction
- thermal diffusivity $\alpha = 1\times10^{-6}\; m^2/s$
- initial temperature of the half-space $T_{init} = 1300^oC$
- surface temperature $T_{surf}=20^oC$

In [ ]:
# Set parameters
L =      # DEFINE DEPTH OF HALF-SPACE HERE [m]!
nz =     # DEFINE NUMBER OF GRIDPOINTS NZ HERE! 
dz = L / (nz - 1)  # spatial distance between two consecutive locations [m]
alpha =    #  DEFINE THERMAL DIFFUSIVITY OF ROCK HERE [m^2/s]!

# Define the depth locations [m]
z = numpy.linspace(0.0, L, num=nz)

# Set the initial temperature in the half-space
T_init =  # INITIAL TEMPERATURE OF THE MOLTEN HALF-SPACE [°C]
T0 = numpy.ones(nz)
T0 = T0 * T_init

# COOL THE HALF-SPACE BY SETTING THE TEMPERATURE AT THE FIRST GRIDPOINT
# TO THE SURFACE TEMPERATURE AS DIRICHLET BOUNDARY CONDITION
T_surf =   # SURFACE TEMPERATURE [°C]
T0[0] = T_surf

To solve the 1D heat conduction equation, we are using the forward-time, centered-space **FTCS** discretization. You should work it out on a piece of paper yourself (if you can't do it without looking it up, it means you need to do this more!).

$$
\begin{equation}
\frac{T_{i}^{n+1}-T_{i}^{n}}{\Delta t}=\alpha\frac{T_{i+1}^{n}-2T_{i}^{n}+T_{i-1}^{n}}{\Delta z^2} \notag
\end{equation}
$$

To obtain the temperature at the next time step, $T^{n+1}_i$, from the known information at the current time step, we compute

$$
\begin{equation}
T_{i}^{n+1}=T_{i}^{n}+\frac{\alpha\Delta t}{\Delta z^2}(T_{i+1}^{n}-2T_{i}^{n}+T_{i-1}^{n}) \notag
\end{equation}
$$

In the [first lesson of module 5](https://nbviewer.jupyter.org/github/numerical-mooc/numerical-mooc/blob/master/lessons/02_spacetime/02_03_1DDiffusion.ipynb), we discretized the diffusion equation  with a forward-time, centered-space scheme, subject to the following stability constraint:

$$
\begin{equation}
\alpha \frac{\Delta t}{(\Delta z)^2} \leq \frac{1}{2} \notag
\end{equation}
$$

The following function already implements this numerical scheme:

In [ ]:
def ftcs_cool_lith(T0, nt, nz, dt, dz, alpha):
    """
    Computes and returns the depth-temperature distrubtion (geotherm)    
    for the half-space cooling problem, according to a provided number 
    of time steps, a given initial temperature and thermal diffusivity.
    The diffusion equation is integrated using forward differencing in 
    time and central differencing in space.
    
    Parameters
    ----------
    T0 : numpy.ndarray
        The initial temperature as a 1D array of floats.
    nt : integer
        The number of time steps to compute.
    nz : integer
        The number of spatial grid points.    
    dt : float
        The time-step size to integrate.
    dz : float
        The distance between two consecutive locations.
    alpha : float
        The thermal diffusivity of the rock.
    
    Returns
    -------
    T : numpy.ndarray
        The geotherm as a 1D array of floats.
    """
    
    T = T0.copy()
    sigma = alpha * dt / dz**2
    for n in range(nt): # loop over time steps
        Tn = T.copy()
        for i in range(1,nz-1): # loop over spatial grid
            T[i] = (Tn[i] + sigma * (Tn[i+1] - 2.0 * Tn[i] + Tn[i-1]))
    return T

We are all set to run! Lets compute the temperature profile after **10 Ma years**.
First, you have to define the maximum cooling time **Tmax in Million years [Ma]**. To get correct results, do not forget to convert the units of Tmax from [Ma] to [s].

Next, we define a time step `dt` that satisfies the stability constraint and calculate the number of time steps `nt` for the given `Tmax` and `dt`. Compute the temperature profile after 10 Ma years using the FD code `T10Ma` and the analytical solution `T10Ma_an`

In [ ]:
# DEFINE MAXIMUM COOLING TIME in [Ma]
Tmax =  

# CONVERT Tmax UNITS [Ma] -> [s] HERE!
Tmax =

# Set the time-step size based on CFL limit.
sigma = 0.5
dt = sigma * dz**2 / alpha  # time-step size
nt = (int)(Tmax/dt)         # number of time steps

# Compute the temperature profile in the lithosphere and asthenosphere
T10Ma = ftcs_cool_lith(T0, nt, nz, dt, dz, alpha)

# COMPUTE ANALYTICAL SOLUTION EQ.(1) HERE!
T10Ma_an = 

Compute the temperature profile after **60 Ma years**

In [ ]:
# DEFINE MAXIMUM COOLING TIME in [Ma]
Tmax = 

# CONVERT Tmax UNITS [Ma] -> [s] HERE!
Tmax =

# Set the time-step size based on CFL limit.
sigma = 0.5
dt = sigma * dz**2 / alpha  # time-step size
nt = (int)(Tmax/dt)         # number of time steps

# Compute the temperature profile in the lithosphere and asthenosphere
T60Ma = ftcs_cool_lith(T0, nt, nz, dt, dz, alpha)

# COMPUTE ANALYTICAL SOLUTION EQ.(1) HERE!
T60Ma_an =

Compute the temperature profile after **100 Ma years**

In [ ]:
# DEFINE MAXIMUM COOLING TIME in [Ma]
Tmax = 

# CONVERT Tmax UNITS [Ma] -> [s] HERE!
Tmax =

# Set the time-step size based on CFL limit.
sigma = 0.5
dt = sigma * dz**2 / alpha  # time-step size
nt = (int)(Tmax/dt)         # number of time steps

# Compute the temperature profile in the lithosphere and asthenosphere
T100Ma = ftcs_cool_lith(T0, nt, nz, dt, dz, alpha)

# COMPUTE ANALYTICAL SOLUTION EQ.(1) HERE!
T100Ma_an =

Plot the initial and different temperature profiles of the numerical and analytical solutions. Describe, compare and interpret the results ...

In [ ]:
# Plot the temperature profiles of the half-space after Tmax = 10, 60, 100 Ma
pyplot.figure(figsize=(8.0, 6.0))
pyplot.ylabel('Depth [km]')
pyplot.xlabel('Temperature [C]')
pyplot.grid()

# Initial temperature profile
pyplot.plot(T0, z/1000, color='C6', linestyle='-', linewidth=3, label='Tmax = 0 Ma')

# Temperature profile after 10 Ma
pyplot.plot(T10Ma, z/1000, color='C0', linestyle='-', linewidth=3, label='Tmax = 10 Ma')
pyplot.plot(T10Ma_an, z/1000, color='C1', linestyle='--', label='Tmax_an = 10 Ma', linewidth=2)

# Temperature profile after 60 Ma
pyplot.plot(T60Ma, z/1000, color='C2', linestyle='-', linewidth=3, label='Tmax = 60 Ma')
pyplot.plot(T60Ma_an, z/1000, color='C3', linestyle='--', label='Tmax_an = 60 Ma', linewidth=2)

# Temperature profile after 100 Ma
pyplot.plot(T100Ma, z/1000, color='C4', linestyle='-', linewidth=3, label='Tmax = 100 Ma')
pyplot.plot(T100Ma_an, z/1000, color='C5', linestyle='--', label='Tmax_an = 100 Ma', linewidth=2)

pyplot.legend()
pyplot.ylim(0.0, L/1000)
pyplot.xlim(100.0, 1500.0)
pyplot.gca().invert_yaxis()

##### Exercise 2

The boundary between the rigid [lithosphere](https://en.wikipedia.org/wiki/Lithosphere) and the partially molten [asthenosphere](https://en.wikipedia.org/wiki/Asthenosphere)
is defined by the $1100^oC$ isotherm. Modify the code `ftcs_lith_thickness` below to compute the thickness of the lithosphere at each time step and return the lithospheric thickness `d_lith` and times `t` 

In [ ]:
def ftcs_lith_thickness(T0, nt, nz, dt, dz, alpha):
    """
    Computes and returns the depth-temperature distrubtion (geotherm)    
    for the half-space cooling problem, according to a provided number 
    of time steps, a given initial temperature and thermal diffusivity.
    The diffusion equation is integrated using forward differencing in 
    time and central differencing in space.
    
    Parameters
    ----------
    T0 : numpy.ndarray
        The initial temperature as a 1D array of floats.
    nt : integer
        The number of time steps to compute.
    nz : integer
        The number of spatial grid points.    
    dt : float
        The time-step size to integrate.
    dz : float
        The distance between two consecutive locations.
    alpha : float
        The thermal diffusivity of the rock.
    
    Returns
    -------
    d_lith : numpy.ndarray
        The lithosphere thickness as a 1D array of floats.
    t : numpy.ndarray
        Time as a 1D array of floats.    
    """
    
    T = T0.copy()
    sigma = alpha * dt / dz**2
    for n in range(nt): # loop over time steps
        Tn = T.copy()
        
        for i in range(1,nz-1): # loop over spatial grid
            T[i] = (Tn[i] + sigma * (Tn[i+1] - 2.0 * Tn[i] + Tn[i-1]))
            
        # ESTIMATE THE LITHOSPHERE THICKNESS BASED ON THE 1100°C ISOTHERM HERE!
        TLAB = 1100.

        
    return d_lith, t

Compute the evolution of the lithosphere thickness up to Tmax = 100 Ma

In [ ]:
# Compute lithosphere thickness for Tmax = 100 Ma
Tmax =

# CONVERT Tmax UNITS [Ma] -> [s] HERE!
Tmax =

# Set the time-step size based on CFL limit.
sigma = 0.5
dt = sigma * dz**2 / alpha  # time-step size
nt = (int)(Tmax/dt)         # number of time steps

# initialize arrays for lithosphere thickness d_lith and time t
d_lith = numpy.zeros(nt)
t = numpy.zeros(nt)

# Compute the thickness of the lithosphere d_lith at times t
d_lith, t = ftcs_lith_thickness(T0, nt, nz, dt, dz, alpha)

Plot lithosphere depth over time. The thickness of the oceanic lithosphere can be approximated as a thermal boundary layer that thickens with the square root of time

\begin{equation}
h \approx 2 \sqrt{\alpha t} \tag{2}
\end{equation}

Here, $h$ denotes the thickness of the oceanic mantle lithosphere, $\alpha$ is the thermal diffusivity and $t$ is the age of the given part of the lithosphere. Compare and discuss the thickness of the lithosphere based on the FD solution $d_{lith}$ and approximation $h$ from eq. (2) 

In [ ]:
# Plot the lithosphere thickness over time
pyplot.figure(figsize=(8.0, 6.0))
pyplot.ylabel('Lithosphere Thickness [km]')
pyplot.xlabel('Time [Ma]')
pyplot.grid()

# PLOT LITHOSPHERE THICKNESS OVER TIME FROM FD MODELLING HERE!
pyplot.plot(t, d_lith, color='C0', linestyle='-', linewidth=3, label='FD-based dlith')

# COMPUTE LITHOSPHERE THICKNESS OVER TIME FROM EQ.(2) HERE!
h =

# PLOT LITHOSPHERE THICKNESS OVER TIME FROM EQ.(2) HERE!
pyplot.plot(t, h, color='C1', linestyle='-', linewidth=3, label='analytical-based h')

pyplot.legend()

## What we learned:

- How to solve the cooling of a half-space problem using the finite-difference and analytical solution of the 1D heat conduction equation
- Estimation of the lithosphere thickness based on the $1100°C$ isotherm